In [1]:
from argparse import ArgumentParser
from functools import reduce
from pathlib import Path
import os
from typing import Optional

import yaml
from caf.core.data_structures import DVector, IpfTarget
from caf.core.segments import SegmentsSuper
from caf.core.segmentation import Segmentation, SegmentationInput
from caf.core.zoning import TranslationWeighting
import numpy as np
import pandas as pd

os.chdir('..')
from land_use import constants, data_processing
os.chdir('supporting_processes')

C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read in a DVector to IPF
input_path = Path(r'F:\Working\Land-Use\OUTPUTS_full run_final')
file = 'Output P11_EM.hdf'

In [3]:
# load dvector data
dvector = DVector.load(input_path / file)

In [4]:
# add required segmentation for IPF
dvector = dvector.add_segments(new_segs=['status_aps'])

In [5]:
dvector.data.head()

LSOA2021                                                                                                    E01013453  \
accom_h ns_sec adults car_availability children adult_nssec age_9 g economic_status pop_emp soc status_aps              
1       1      1      1                1        1           4     1 1               1       1   1            0.018584   
                                                                                            2   1            0.003041   
                                                                                    2       1   1            0.004393   
                                                                                            2   1            0.003717   
                                                                    2               3       4   2            0.001014   

LSOA2021                                                                                                    E01013454  \
accom_h ns_sec adults car_availability children adult_nssec age_9 g economic_status pop_emp soc status_aps              
1       1      1      1                1        1           4     1 1               1       1   1            0.017155   
                                                                                            2   1            0.002807   
                                                                                    2       1   1            0.004055   
                                                                                            2   1            0.003431   
                                                                    2               3       4   2            0.000936   

LSOA2021                                                                                                    E01013455  \
accom_h ns_sec adults car_availability children adult_nssec age_9 g economic_status pop_emp soc status_aps              
1       1      1      1                1        1           4     1 1               1       1   1            0.173532   
                                                                                            2   1            0.018171   
                                                                                    2       1   1            0.048607   
                                                                                            2   1            0.018171   
                                                                    2               3       4   2            0.009540   

LSOA2021                                                                                                    E01013456  \
accom_h ns_sec adults car_availability children adult_nssec age_9 g economic_status pop_emp soc status_aps              
1       1      1      1                1        1           4     1 1               1       1   1            0.002436   
                                                                                            2   1            0.000399   
                                                                                    2       1   1            0.000576   
                                                                                            2   1            0.000487   
                                                                    2               3       4   2            0.000133   

LSOA2021                                                                                                    E01013457  \
accom_h ns_sec adults car_availability children adult_nssec age_9 g economic_status pop_emp soc status_aps              
1       1      1      1                1        1           4     1 1               1       1   1            0.031165   
                                                                                            2   1            0.005100   
                                                                                    2       1   1            0.007366   
   

In [6]:
# read in the APS targets and store as DVector
target = Path(r'I:\NorMITs Land Use\2023\import\APS 2023 for IPF\Regional-based-targets\preprocessing\APS-24-regional-based-targets_revamp_t01.hdf')
target_dvector = data_processing.read_dvector_data(
    file_path=target,
    geographical_level='RGN2021',
    input_segments=['g', 'status_aps'],
    geography_subset='EM'
)

The input data at I:\NorMITs Land Use\2023\import\APS 2023 for IPF\Regional-based-targets\preprocessing\APS-24-regional-based-targets_revamp_t01.hdf started with 11 columns. Filtering to EM results in 1 columns.
C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\segmentation.py:343: SegmentationWarning: Read in level status_aps is a subset of the segment. If this was not expected check the input segmentation.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\data_structures.py:437: UserWarning: 0 dropped on seg validation.
  warnings.warn(f"{full_sum - cut_sum} dropped on seg validation.")


In [7]:
target_dvector.data.head()

RGN2021       E12000004
g status_aps           
1 1             1269700
  2               55400
  3              599600
2 1             1080300
  2               40500

In [8]:
target_dvector.segmentation.seg_dict

{'g': Segment(name='g', values={1: 'male', 2: 'female'}, exclusions=[Exclusion(other_name='gender_3', exclusions={1: {3}, 2: {2}})], lookups=[]),
 'status_aps': Segment(name='status_aps', values={1: 'Economically active employed', 2: 'Economically active unemployed', 3: 'Economically inactive'}, exclusions=[], lookups=[Exclusion(other_name='economic_status', exclusions={1: {1, 3}, 2: {2, 4}, 3: {5, 6}, -8: {-8}})])}

In [9]:
# try and apply IPF with the subset target
rebalanced_data, rmse = dvector.ipf(
    targets=[IpfTarget(target_dvector)],
    zone_trans_cache=constants.CACHE_FOLDER
)

C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\data_structures.py:437: UserWarning: 0 dropped on seg validation.
  warnings.warn(f"{full_sum - cut_sum} dropped on seg validation.")
C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\data_structures.py:437: UserWarning: 0.0 dropped on seg validation.
  warnings.warn(f"{full_sum - cut_sum} dropped on seg validation.")
C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\data_structures.py:437: UserWarning: 0.0 dropped on seg validation.
  warnings.warn(f"{full_sum - cut_sum} dropped on seg validation.")
C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\data_structures.py:437: UserWarning: 0.0 dropped on seg validation.
  warnings.warn(f"{full_sum - cut_sum} dropped on seg validation.")


Convergence met, returning DVector.


C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\data_structures.py:758: SegmentationWarning: There are subsets in other's segmentation not in self. This will lead to rows present in self being cut in the product. Missing values are {'status_aps': [-8]}
  warnings.warn(
C:\ProgramData\Anaconda3\envs\normits_lu\Lib\site-packages\caf\core\data_structures.py:870: SegmentationWarning: This operation has changed the segmentation of the DVector from ['g', 'status_aps'] to ['g', 'status_aps']. This can happen but it can also be a sign of an error. Check the output DVector.
  warnings.warn(


In [21]:
rebalanced_data.aggregate(segs=['status_aps']).data

LSOA2021,E01013453,E01013454,E01013455,E01013456,E01013457,E01013458,E01013459,E01013460,E01013461,E01013462,...,E01035501,E01035502,E01035503,E01035504,E01035505,E01035506,E01035507,E01035508,E01035509,E01035510
status_aps,,,,,,,,,,,,,,,,,,,,,
-8,363.097525,328.709458,350.960560,348.937733,358.040456,328.709458,503.684031,246.784947,285.218668,312.526839,...,203.294157,80.913097,86.981580,134.518024,279.150186,75.856029,344.892078,532.003615,255.887671,719.115153
1,1176.723751,694.699176,944.726165,876.003037,907.716661,805.647973,771.727540,1005.162802,865.921374,774.327205,...,1054.632762,776.186504,460.836902,539.984196,885.525655,1286.299943,2456.102085,2548.844117,652.810944,1172.598868
2,85.848898,44.808341,40.993193,73.335548,57.517238,34.085689,44.416948,66.287037,21.594424,24.485470,...,109.873906,103.927193,77.217056,70.231465,116.031297,94.331037,131.705224,161.559718,20.265239,30.911348
3,620.678884,308.578686,602.493973,497.407726,394.412455,683.458250,763.522079,463.109771,528.934587,385.941877,...,1301.002041,2104.580489,1468.245961,1058.283725,1788.055116,687.497253,2295.057882,1931.983768,396.270676,343.625524


In [22]:
dvector.aggregate(segs=['status_aps']).data

LSOA2021,E01013453,E01013454,E01013455,E01013456,E01013457,E01013458,E01013459,E01013460,E01013461,E01013462,...,E01035501,E01035502,E01035503,E01035504,E01035505,E01035506,E01035507,E01035508,E01035509,E01035510
status_aps,,,,,,,,,,,,,,,,,,,,,
-8,363.097525,328.709458,350.960560,348.937733,358.040456,328.709458,503.684031,246.784947,285.218668,312.526839,...,203.294157,80.913097,86.981580,134.518024,279.150186,75.856029,344.892078,532.003615,255.887671,719.115153
1,1347.138544,803.562127,1088.510277,1005.086609,1046.951429,926.559535,886.870831,1149.478190,999.519832,890.933048,...,1213.458577,896.236961,528.870012,618.133400,1018.136735,1464.288477,2834.264873,2937.652786,753.185010,1355.587262
2,141.798588,75.028332,68.562420,121.219136,96.362712,56.848742,74.015513,109.461781,36.277950,40.929554,...,183.694533,174.778516,128.615711,116.498244,193.841906,154.740054,221.614950,271.406705,33.990753,52.034958
3,515.684856,255.204318,495.577318,417.111588,327.281404,562.031883,637.147329,386.630147,442.889074,316.221925,...,1081.821912,1744.760312,1225.766619,886.664546,1491.530226,582.819132,1901.058807,1605.269805,330.436395,284.472930
